# Contribution Analysis Tutorial

This short tutorial shows how to run contribution analysis and generate plots with different settings.

In [15]:
%load_ext autoreload
%autoreload 2

# Import required functions
from mescal.contribution_analysis import process_contribution_data
from mescal.plot import plot_contribution_analysis
import sys
import os

# Add parent directory to path to import streamlit_app from root
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from streamlit_app import launch_streamlit_app


2026-02-09 14:15:43.555 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-09 14:15:43.556 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-09 14:15:43.557 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-09 14:15:43.557 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-09 14:15:43.558 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-09 14:15:43.558 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-09 14:15:43.558 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-09 14:15:43.559 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Setup: Define input file paths

Set paths to your contribution data, impact scores, and unit conversion files.

In [16]:
# Define paths to input data files (adjust these to your data location)
contrib_emissions_path = '../testing/contrib_analysis_emissions.csv'
contrib_processes_path = '../testing/contrib_analysis_processes.csv'
impact_scores_path = '../testing/impact_scores_reg.csv'
unit_conversion_path = '../testing/unit_conversion.csv'
output_dir = '../testing/Contribution_Analysis'

## 1) Default Settings: Process and Plot

This example shows the basic workflow with default settings. The commented-out export option shows how to export to Excel.


In [17]:
# Process the contribution data with minimal parameters
# - All impacts and activity types will be included
# - Results are NOT exported to Excel by default
grouped_df, unit_type_groups_dict = process_contribution_data(
    contrib_data_path=contrib_emissions_path,
    impact_scores_path=impact_scores_path,
    unit_conversion_path=unit_conversion_path,
    contribution_type='emissions',
    output_dir=output_dir,
    export_excel=False,  # Set to True to save results as Excel file
)


In [18]:
# Generate plots for all impact categories with minimal parameters
# Default behavior: includes all data and uses standard styling

# plot_contribution_analysis(
#     df=grouped_df,
#     unit_type_groups_dict=unit_type_groups_dict,
#     output_dir=output_dir,
#     contribution_type='emissions',
# )

## 2) Advanced Example: All Customization Options

This example demonstrates all available settings with inline comments explaining each parameter.


In [19]:
# Process contribution data with all available options
grouped_df_custom, unit_type_groups_dict_custom = process_contribution_data(
    contrib_data_path=contrib_processes_path,  # Path to CSV file (processes or emissions)
    impact_scores_path=impact_scores_path,  # Impact assessment scores
    unit_conversion_path=unit_conversion_path,  # Unit conversion factors
    contribution_type='processes',  # Type of analysis: 'processes' or 'emissions'
    output_dir=output_dir,  # Directory to save results
    export_excel=False,  # Uncomment line below to save results as Excel file
    # export_excel=True,  # Set to True to export processed data as Excel
)


In [ ]:
# Generate customized plots with all available parameters
plot_contribution_analysis(
    df=grouped_df_custom,  # Processed dataframe from process_contribution_data()
    unit_type_groups_dict=unit_type_groups_dict_custom,  # Unit groups dictionary
    output_dir=output_dir,  # Directory to save plot images
    contribution_type='processes',  # Type of analysis: 'processes' or 'emissions'
    # Optional filtering parameters:
    impact_categories=["('EF v3.1 regionalized', 'acidification', 'accumulated exceedance (AE)')"],  # Set to list of strings to filter specific impacts, e.g., ["('EF v3.1 regionalized', 'acidification', 'accumulated exceedance (AE)')"], or None for all
    esm_keys=None,  # Set to list of ESM units to filter, e.g., ['year'], or None for all
    # Plot appearance parameters:
    threshold=0.2,  # Minimum contribution share to display (0-1). Lower values show more detail
    cell_size=0.6,  # Height of each cell in the heatmap (in inches)
    min_fig_width=8,  # Minimum figure width in inches
    min_fig_height=5,  # Minimum figure height in inches
    dpi=150,  # Dots per inch for saved plots. Higher = better quality but larger file size
)

/Users/achuat/Desktop/PhD/Research/MESCAL/mescal/mescal/plot.py:872: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_cat = pd.concat([df_cat, Others_share], ignore_index=True)


## 3) Using the Streamlit App

Launch the interactive Streamlit application for a GUI-based workflow with filtering capabilities.


In [ ]:
# Alternative: Launch with both emissions AND processes files for switching between them in the app
launch_streamlit_app(
    contrib_data_path_emissions=contrib_emissions_path,  # Path to emissions CSV
    contrib_data_path_processes=contrib_processes_path,  # Path to processes CSV
    impact_scores_path=impact_scores_path,  # Impact scores CSV
    unit_conversion_path=unit_conversion_path,  # Unit conversion CSV
)